In [12]:
import sys
sys.path.append("../")
import os
import multiprocessing
from multiprocessing import Pool, cpu_count
import logging
import numpy as np

In [9]:
CHECKFILES=10
MAX_RAM=30 * 2**30 # gb * fac

THREAD_COUND=max(1,int(cpu_count()-10))
DATA_FOLDER="/media/julian/ex1ext4_1/temp/"
MOLPROP_FOLDER=os.path.join(DATA_FOLDER,"CID-PROPS")
PROPVEC_FOLDER=os.path.join(DATA_FOLDER,"prop_vecs")
LOG_FOLDER=os.path.join(DATA_FOLDER,"logs")

os.makedirs(LOG_FOLDER,exist_ok=True)
os.makedirs(MOLPROP_FOLDER,exist_ok=True)
os.makedirs(PROPVEC_FOLDER,exist_ok=True)

logger = logging.getLogger("ecdf_3")
logger.setLevel(logging.DEBUG)
for h in list(logger.handlers):
    logger.removeHandler(h)

fh = logging.FileHandler(os.path.join(LOG_FOLDER,"ecdf_3.log"))
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)

In [10]:
mdt_files=np.array([os.path.join(MOLPROP_FOLDER,f)for f in os.listdir(MOLPROP_FOLDER) if f.endswith(".mdt")])
np.random.shuffle(mdt_files)
len(mdt_files)

1957547

In [11]:
item_sizes={}
item_shapes={}
item_types={}
for mdtf in mdt_files[:CHECKFILES]:
    for k,v in np.load(mdtf).items():
        if k not in item_sizes:
            item_sizes[k]=[]
        if k not in item_shapes:
            item_shapes[k]=[]
        if k not in item_types:
            item_types[k]=[]
        item_sizes[k].append(v.size*v.itemsize)
        item_shapes[k].append(v.shape)
        item_types[k].append(v.dtype)
        
n_files = len(mdt_files)

for k in list(item_sizes.keys()):
    item_sizes[k]=int(np.median(item_sizes[k]))*n_files

for k in list(item_shapes.keys()):
    item_shapes[k]=np.median(item_shapes[k],0).astype(int)
    
item_types={k:max(set(v), key = v.count) for k, v in item_types.items()} 



for b,s in list(item_sizes.items()):
    f=os.path.join(PROPVEC_FOLDER,b+".npy")
    f_size=0
    if os.path.exists(f):
        f_size=os.path.getsize(f)
    sp=min(f_size,s)/max(f_size,s)

    if sp<0.95:
        pass
    else:
        del item_sizes[b]

item_sizes = {k: v for k, v in sorted(item_sizes.items(), key=lambda item: item[1])}

blocks=[]
remaining_ram=-1#MAX_RAM
block_index=-1
block_sizes=[]
for n,az in item_sizes.items():
    if remaining_ram - az<0:
        block_index+=1
        blocks.append([])
        block_sizes.append(0)
        remaining_ram=MAX_RAM
    blocks[block_index].append(n)
    remaining_ram -=az
    block_sizes[block_index]+=az
    
len(blocks),[len(b) for b in blocks],n_files,np.array(block_sizes)/2**30

(0, [], 1957547, array([], dtype=float64))

In [ ]:
def npload(f):
        return dict(np.load(f))
    
def load_block(block):    
    data={b:np.zeros((n_files,*item_shapes[b]),dtype=item_types[b]) for b in block}
    deltaprint=10**(max(0,int(np.log10(len(mdt_files)/THREAD_COUND)))-2)
    idx={b:0 for b in block}
    i=0
    with multiprocessing.Pool(processes=THREAD_COUND) as pool: # or more
        for d in pool.imap_unordered(npload, mdt_files,100):
            if i%deltaprint==0:
                logger.debug(i)
            i+=1
            try:
                for b in block:
                    data[b][idx[b]]=d[b]
    #                print(d[b],data[b][idx[b]])
                    idx[b]+=1
            except Exception:
                continue
    for b in block:
        logger.info(i)
        np.save(os.path.join(PROPVEC_FOLDER,b),np.stack(data[b]))
for b in blocks:
    load_block(b)

In [ ]:
%timeit np.load(mdt_files[np.random.randint(len(mdt_files))])